In [1]:
import pandas as pd
import numpy as np

In [2]:
vehicles_df = pd.read_csv('data/vehicles.csv')
demand_df = pd.read_csv('data/demand.csv')

In [3]:
vehicles_df

,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance
0,BEV_S1_2023,BEV,S1,2023,187000,102000,D1
1,BEV_S1_2024,BEV,S1,2024,177650,102000,D1
2,BEV_S1_2025,BEV,S1,2025,168767,102000,D1
3,BEV_S1_2026,BEV,S1,2026,160329,102000,D2
4,BEV_S1_2027,BEV,S1,2027,152312,102000,D2
...,...,...,...,...,...,...,...
187,LNG_S3_2034,LNG,S3,2034,209208,73000,D4
188,LNG_S3_2035,LNG,S3,2035,215484,73000,D4
189,LNG_S3_2036,LNG,S3,2036,221948,73000,D4
190,LNG_S3_2037,LNG,S3,2037,228607,73000,D4


In [4]:
demand_df

,Year,Size,Distance,Demand (km)
0,2023,S1,D1,869181
1,2023,S1,D2,2597094
2,2023,S1,D3,3292011
3,2023,S1,D4,414315
4,2023,S2,D1,995694
...,...,...,...,...
251,2038,S4,D4,2446
252,2038,S3,D1,3229278
253,2038,S3,D2,3297618
254,2038,S3,D3,1448550


In [5]:
def allocate_vehicles(demand_df, vehicles_df, year):
    # Mapping for categorical distance values
    distance_mapping = {"D1": 1, "D2": 2, "D3": 3, "D4": 4}
    
    # Convert categorical distance to numeric ranks
    vehicles_df['Distance_categorical'] = vehicles_df['Distance'].map(distance_mapping)
    demand_df['Distance_categorical'] = demand_df['Distance'].map(distance_mapping)

    demand_df = demand_df[demand_df['Year'] == year].copy() # Filter relevant year
    available_vehicles = vehicles_df[vehicles_df['Year'] <= year]  # Vehicles available in that year

    def find_allocations(row):
        size_needed = row['Size']
        distance_needed = row['Distance_categorical']
        
        allocated_vehicles = available_vehicles[
            (available_vehicles['Size'] == size_needed) & 
            (available_vehicles['Distance_categorical'] >= distance_needed)
        ]['ID'].tolist() 
        
        return allocated_vehicles

    demand_df['Allocation'] = demand_df.apply(find_allocations, axis=1)  # Apply function row-wise
    
    return demand_df


In [6]:
allocation_df = allocate_vehicles(demand_df, vehicles_df, 2023)
allocation_df.to_csv('data/allocation_result.csv', index=False)

In [7]:
allocation_df

,Year,Size,Distance,Demand (km),Distance_categorical,Allocation
0,2023,S1,D1,869181,1,"[BEV_S1_2023, Diesel_S1_2023, LNG_S1_2023]"
1,2023,S1,D2,2597094,2,"[Diesel_S1_2023, LNG_S1_2023]"
2,2023,S1,D3,3292011,3,"[Diesel_S1_2023, LNG_S1_2023]"
3,2023,S1,D4,414315,4,"[Diesel_S1_2023, LNG_S1_2023]"
4,2023,S2,D1,995694,1,"[BEV_S2_2023, Diesel_S2_2023, LNG_S2_2023]"
5,2023,S2,D2,1383196,2,"[Diesel_S2_2023, LNG_S2_2023]"
6,2023,S2,D3,778008,3,"[Diesel_S2_2023, LNG_S2_2023]"
7,2023,S2,D4,133677,4,"[Diesel_S2_2023, LNG_S2_2023]"
8,2023,S4,D1,14576,1,"[BEV_S4_2023, Diesel_S4_2023, LNG_S4_2023]"
9,2023,S4,D2,754717,2,"[Diesel_S4_2023, LNG_S4_2023]"


In [8]:
df_exploded = allocation_df.explode("Allocation")

In [9]:
# Display the transformed DataFrame
print(df_exploded)

    Year Size Distance  Demand (km)  Distance_categorical      Allocation
0   2023   S1       D1       869181                     1     BEV_S1_2023
0   2023   S1       D1       869181                     1  Diesel_S1_2023
0   2023   S1       D1       869181                     1     LNG_S1_2023
1   2023   S1       D2      2597094                     2  Diesel_S1_2023
1   2023   S1       D2      2597094                     2     LNG_S1_2023
2   2023   S1       D3      3292011                     3  Diesel_S1_2023
2   2023   S1       D3      3292011                     3     LNG_S1_2023
3   2023   S1       D4       414315                     4  Diesel_S1_2023
3   2023   S1       D4       414315                     4     LNG_S1_2023
4   2023   S2       D1       995694                     1     BEV_S2_2023
4   2023   S2       D1       995694                     1  Diesel_S2_2023
4   2023   S2       D1       995694                     1     LNG_S2_2023
5   2023   S2       D2      1383196   

In [10]:
df = pd.merge(df_exploded, vehicles_df, how='inner', left_on='Allocation', right_on='ID')

In [11]:
df['Num_Vehicles'] = np.ceil(df['Demand (km)']/df['Yearly range (km)'])

In [12]:
df["Distance_per_vehicle(km)"] = df["Demand (km)"] / df["Num_Vehicles"]

In [13]:
df["Operating Year"] = 2023

In [14]:
df.columns

Index(['Year_x', 'Size_x', 'Distance_x', 'Demand (km)',
       'Distance_categorical_x', 'Allocation', 'ID', 'Vehicle', 'Size_y',
       'Year_y', 'Cost ($)', 'Yearly range (km)', 'Distance_y',
       'Distance_categorical_y', 'Num_Vehicles', 'Distance_per_vehicle(km)',
       'Operating Year'],
      dtype='object')

In [15]:
df2 = df[['Operating Year', 'ID','Num_Vehicles', 'Distance_y', 'Distance_per_vehicle(km)']].copy()

In [16]:
vehicle_fuels_df = pd.read_csv('data/vehicles_fuels.csv')

In [17]:
df = pd.merge(df2, vehicle_fuels_df, how='left', left_on='ID', right_on='ID')

In [18]:
df.drop('Consumption (unit_fuel/km)', axis=1, inplace=True)

In [19]:
df.rename(columns={'Distance_y': 'Distance_bucket'}, inplace=True)

In [20]:
df

,Operating Year,ID,Num_Vehicles,Distance_bucket,Distance_per_vehicle(km),Fuel
0,2023,BEV_S1_2023,9.0,D1,96575.666667,Electricity
1,2023,Diesel_S1_2023,9.0,D4,96575.666667,B20
2,2023,Diesel_S1_2023,9.0,D4,96575.666667,HVO
3,2023,LNG_S1_2023,9.0,D4,96575.666667,LNG
4,2023,LNG_S1_2023,9.0,D4,96575.666667,BioLNG
...,...,...,...,...,...,...
63,2023,LNG_S3_2023,14.0,D4,71604.714286,BioLNG
64,2023,Diesel_S3_2023,3.0,D4,68475.333333,B20
65,2023,Diesel_S3_2023,3.0,D4,68475.333333,HVO
66,2023,LNG_S3_2023,3.0,D4,68475.333333,LNG


In [12]:
# Define column names
columns = ["Operating Year","ID","Num_Vehicles","Type","Fuel","Distance_bucket","Distance_per_vehicle(km)"]

# Create an empty DataFrame
empty_df = pd.DataFrame(columns=columns)

# Display the empty DataFrame
print(empty_df)

Empty DataFrame
Columns: [Operating Year, ID, Num_Vehicles, Type, Fuel, Distance_bucket, Distance_per_vehicle(km)]
Index: []


In [22]:
from utilities.my_sql_operations import MySQLOperations
import pandas as pd

In [23]:
def carbon_emmissions(fleet_details: pd.DataFrame, op_year: int):
        """
        Calculates total carbon emissions of the fleet
        """     
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT fuel, emissions_co2_per_unit_fuel FROM fuels WHERE year = {op_year}"""
        fuel_data, columns = my_sql_operations.fetch_data(query) 
        fuel_df = pd.DataFrame(fuel_data, columns=columns)
        
        query = f"""SELECT * FROM vehicles_fuels"""
        vehicles_fuels_data, columns = my_sql_operations.fetch_data(query)
        vehicles_fuels_df = pd.DataFrame(vehicles_fuels_data, columns=columns)

        merged_df = pd.merge(
            pd.merge(fleet_details, fuel_df, left_on='Fuel', right_on='fuel', how='left'),
            vehicles_fuels_df, left_on=['ID', 'Fuel'], right_on=['id', 'fuel'], how='left'
        )
        
        merged_df['carbon_emissions'] = (
            merged_df['Distance_per_vehicle(km)'] * 
            merged_df['Num_Vehicles'] * 
            merged_df['emissions_co2_per_unit_fuel'] *
            merged_df['consumption_unitfuel_per_km']
        )
        return merged_df['carbon_emissions']

In [25]:
df['carbon_emissions'] = carbon_emmissions(df, 2023)

In [26]:
df

,Operating Year,ID,Num_Vehicles,Distance_bucket,Distance_per_vehicle(km),Fuel,carbon_emissions
0,2023,BEV_S1_2023,9.0,D1,96575.666667,Electricity,0.000000
1,2023,Diesel_S1_2023,9.0,D4,96575.666667,B20,590940.618580
2,2023,Diesel_S1_2023,9.0,D4,96575.666667,HVO,94521.655392
3,2023,LNG_S1_2023,9.0,D4,96575.666667,LNG,357652.204413
4,2023,LNG_S1_2023,9.0,D4,96575.666667,BioLNG,54326.078282
...,...,...,...,...,...,...,...
63,2023,LNG_S3_2023,14.0,D4,71604.714286,BioLNG,62476.536420
64,2023,Diesel_S3_2023,3.0,D4,68475.333333,B20,140333.054421
65,2023,Diesel_S3_2023,3.0,D4,68475.333333,HVO,22446.439106
66,2023,LNG_S3_2023,3.0,D4,68475.333333,LNG,84179.732177


In [30]:
def yearly_insurance_cost(fleet_details: pd.DataFrame):
        """
        Returns Insurance cost for the operating year
        """
        total_fleet_insurance_cost = 0
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT id, year, cost FROM vehicles"""
        purchase_year_data, columns = my_sql_operations.fetch_data(query) 
        purchase_year_df = pd.DataFrame(purchase_year_data, columns=columns)
        
        query = f"""SELECT end_of_year, insurance_cost_percent FROM cost_profiles"""
        insurance_cost_data, columns = my_sql_operations.fetch_data(query)
        insurance_cost_df = pd.DataFrame(insurance_cost_data, columns=columns)
        
        eoy_df = pd.merge(fleet_details, purchase_year_df, left_on=['ID'], right_on=['id'], how='left')
                
        eoy_df['End_of_year'] = (
            eoy_df['Operating Year'] - eoy_df['year'] + 1
        )
        
        merged_df = pd.merge(eoy_df, insurance_cost_df, left_on='End_of_year', right_on='end_of_year', how='left')
        
        merged_df['insurance_cost'] = (
            ((merged_df['insurance_cost_percent']/100) * merged_df['cost']) * merged_df['Num_Vehicles'] 
        )
        return merged_df['insurance_cost']
            
def yearly_maintenance_cost(fleet_details: pd.DataFrame):
        """
        Returns Maintenance cost for the operating year
        """
        total_fleet_maintainance_cost = 0
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT id, year, cost FROM vehicles"""
        purchase_year_data, columns = my_sql_operations.fetch_data(query) 
        purchase_year_df = pd.DataFrame(purchase_year_data, columns=columns)
        
        query = f"""SELECT end_of_year, maintenance_cost_percent FROM cost_profiles"""
        maintenance_cost_data, columns = my_sql_operations.fetch_data(query)
        maintenance_cost_df = pd.DataFrame(maintenance_cost_data, columns=columns)
        
        eoy_df = pd.merge(fleet_details, purchase_year_df, left_on=['ID'], right_on=['id'], how='left')
                
        eoy_df['End_of_year'] = (
            eoy_df['Operating Year'] - eoy_df['year'] + 1
        )
        
        merged_df = pd.merge(eoy_df, maintenance_cost_df, left_on='End_of_year', right_on='end_of_year', how='left')
        
        merged_df['maintenance_cost'] = (
            ((merged_df['maintenance_cost_percent']/100) * merged_df['cost']) * merged_df['Num_Vehicles'] 
        )
        return merged_df['maintenance_cost']
          
def yearly_fuel_cost(fleet_details: pd.DataFrame, op_year: int):
        """
        Returns yearly fuel cost
        """
        yearly_fuel_cost = 0
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT fuel, cost_per_unit_fuel FROM fuels WHERE year = {op_year}"""
        fuel_cost_data, columns = my_sql_operations.fetch_data(query) 
        fuel_cost_df = pd.DataFrame(fuel_cost_data, columns=columns)
        
        query = f"""SELECT * FROM vehicles_fuels"""
        fuel_consumption_data, columns = my_sql_operations.fetch_data(query)
        fuel_consumption_df = pd.DataFrame(fuel_consumption_data, columns=columns)
        
        merged_df = pd.merge(
            pd.merge(fleet_details, fuel_cost_df, left_on='Fuel', right_on='fuel', how='left'),
            fuel_consumption_df, left_on=['ID', 'Fuel'], right_on=['id', 'fuel'], how='left'
        )
        
        merged_df['fuel_costs'] = (
            merged_df['Distance_per_vehicle(km)'] * 
            merged_df['Num_Vehicles'] * 
            merged_df['consumption_unitfuel_per_km'] *
            merged_df['cost_per_unit_fuel']
        )
        return merged_df['fuel_costs']

In [31]:
df['insurance_cost'] = yearly_insurance_cost(df)
df['maintenance_cost'] = yearly_maintenance_cost(df)
df['fuel_costs'] = yearly_fuel_cost(df, 2023)


In [32]:
df

,Operating Year,ID,Num_Vehicles,Distance_bucket,Distance_per_vehicle(km),Fuel,carbon_emissions,insurance_cost,maintenance_cost,fuel_costs
0,2023,BEV_S1_2023,9.0,D1,96575.666667,Electricity,0.000000,84150.00,16830.00,148871.244349
1,2023,Diesel_S1_2023,9.0,D4,96575.666667,B20,590940.618580,38250.00,7650.00,236649.175940
2,2023,Diesel_S1_2023,9.0,D4,96575.666667,HVO,94521.655392,38250.00,7650.00,351688.154385
3,2023,LNG_S1_2023,9.0,D4,96575.666667,LNG,357652.204413,45000.00,9000.00,145527.141139
4,2023,LNG_S1_2023,9.0,D4,96575.666667,BioLNG,54326.078282,45000.00,9000.00,156159.894928
...,...,...,...,...,...,...,...,...,...,...
63,2023,LNG_S3_2023,14.0,D4,71604.714286,BioLNG,62476.536420,105795.20,21159.04,179588.324270
64,2023,Diesel_S3_2023,3.0,D4,68475.333333,B20,140333.054421,17727.15,3545.43,56198.035203
65,2023,Diesel_S3_2023,3.0,D4,68475.333333,HVO,22446.439106,17727.15,3545.43,83516.805846
66,2023,LNG_S3_2023,3.0,D4,68475.333333,LNG,84179.732177,22670.40,4534.08,34252.370360


In [ ]:
df.to_csv('')